In [30]:
#Install package in anaconda prompt 
# pip install requests requests[socks]

#Load packages and prepare the header statement
from bs4 import BeautifulSoup
import re
import requests
import time
import random
import csv
import datetime 
import numpy as np
import os, sys
import os.path
import urllib.request
from stem import Signal
from stem.control import Controller

#Prepare Header for www.funda.nl
headers = {
    'Connection': 'keep-alive',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
}

In [2]:
#Start tor.exe and Privoxy.exe to enable IP Rotation

class ConnectionManager:
    def __init__(self):
        self.new_ip = "0.0.0.0"
        self.old_ip = "0.0.0.0"
        self.new_identity()

    @classmethod
    def _get_connection(self):
        """
        TOR new connection
        """
        with Controller.from_port(port=9051) as controller:
            controller.authenticate(password="1234")
            controller.signal(Signal.NEWNYM)
            controller.close()

    @classmethod
    def _set_url_proxy(self):
        """
        Request to URL through local proxy
        """
        proxy_support = urllib.request.ProxyHandler({"http": "127.0.0.1:8118"})
        opener = urllib.request.build_opener(proxy_support)
        urllib.request.install_opener(opener)

    @classmethod
    def request(self, url):
        """
        TOR communication through local proxy
        :param url: web page to parser
        :return: request
        """
        try:
            self._set_url_proxy()
            request = urllib.request.Request(url, None, {
                'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) "
                              "AppleWebKit/535.11 (KHTML, like Gecko) "
                              "Ubuntu/10.10 Chromium/17.0.963.65 "
                              "Chrome/17.0.963.65 Safari/535.11"})
            request = urllib.request.urlopen(request)
            return request
        except urllib.request.HTTPError:
            return e.message

    def new_identity(self):
        """
        new connection with new IP
        """
        # First Connection
        if self.new_ip == "0.0.0.0":
            self._get_connection()
            self.new_ip = self.request("http://icanhazip.com/").read()
        else:
            self.old_ip = self.new_ip
            self._get_connection()
            self.new_ip = self.request("http://icanhazip.com/").read()

        seg = 0

        # If we get the same ip, we'll wait 5 seconds to request a new IP
        while self.old_ip == self.new_ip:
            time.sleep(5)
            seg += 5
            print ("Waiting to obtain new IP: %s Seconds" % seg                                   ,  end="\r")
            self.new_ip = self.request("http://icanhazip.com/").read()
        
cm = ConnectionManager()

In [49]:
#Get the list of links from the search page for all search pages, saved in a csv file per row. 
#Number of pages= 8443
#Information is always stored in linklist, even when the process gets terminated 

try:
    not(os.path.exists('houselinks.csv'))
except:  
    os.remove('houselinks.csv')

#Set Starting IP Adress:
cm.new_identity()
    
with open('houselinks.csv', 'w') as csvfile:
    url_link_list = csv.writer(csvfile)    
    linklist = []
    averagetime = []
    
    ###IMPORTANT### Set the number to scrape
    numbertoscrape = 8443
    
    #Start of home link scraper
    for i in range(2,numbertoscrape):
        try:
            start = datetime.now()
            response = requests.get('http://www.funda.nl/koop/heel-nederland/p'+str(i)+'/', headers=headers).text
            Funda = BeautifulSoup(response, 'html.parser')
            Houses = Funda.find_all('div', {'class': 'search-result-header'})
            for tag in Houses:
                link = [tag.a.get('href')]
                for tag in link:
                    linklist = linklist + link

            #IP Randomizer
            if i % 50 == 0:
                cm.new_identity()

            #Calculate Time
            end = datetime.now()
            averagetime += [(end - start).total_seconds()]
            meantime = np.mean(averagetime)
            timeleft = str(datetime.timedelta(seconds=(numbertoscrape-i * meantime)))

            #print percentage and time remaining
            print(str(round((i/numbertoscrape)*100,2))+'% Complete' + ', Time remaining: ' + str(timeleft)+ ", Scraping on IP adress "+ str(cm.new_ip)[2:15],  end="\r")
        except:
            cm.new_identity()
            
    url_link_list.writerow(linklist)
    
print("100.0% Complete, Scraping process complete in"+str(time.strftime("%H:%M:%S", time.gmtime(sum(averagetime)))))

100.0% Complete, Scraping process complete in04:20:25 185.38.14.215


In [3]:
#Prepare the list of url's to houses in the variable linkelist
with open('houselinks.csv', 'r') as f:
    linkelist = list(csv.reader(f))[0]

In [4]:
#Check the number of houses in the linkelist file
len(linkelist)

126488

In [5]:
#Home site scraper, developed to scrape all elements of one particular home. 
elementschar = ['Oorspronkelijke vraagprijs', 'Aangeboden sinds','Soort bouw', 'Soort dak', 'Aantal kamers',
                'Aantal badkamers', 'Badkamervoorzieningen', 'Aantal woonlagen', 'Voorzieningen',
                'Verwarming', 'Cv-ketel', 'Eigendomssituatie', 'Ligging', 'Ligging tuin', 'Tuin', 'Balkon/dakterras',
                'Schuur/berging', 'Soort garage', 'Soort parkeergelegenheid']
elementsint = ['Vraagprijs', 'Bouwjaar', 'Woonoppervlakte', 'Overige inpandige ruimte', 'Gebouwgebonden buitenruimte',
                  'Externe bergruimte', 'Perceeloppervlakte', 'Inhoud', 'Capagiteit']

def scrapehouse(i):
    dic = {} 
    for x in elementschar:
        try:
            dic[str(x)] = House.find('div', class_='object-primary').find('dt', text=x).find_next_sibling('dd').text[:-2]
        except:
            dic[str(x)] = ""      
    for x in elementsint:
        try:
            dic[str(x)] = int(''.join(re.findall('\d+', House.find('div', class_='object-primary').find('dt', text=x).find_next_sibling('dd').text)))
        except:
            dic[str(x)] = ""     
    try:
        dic['Isolatie'] = House.find('div', class_='object-primary').find('dt', text='Isolatie').find_next_sibling('dd').text[:-12]
    except:
        dic['Isolatie'] = ""
    try:
        dic['Voorlopig energielabel'] = House.find('div', class_='object-primary').find('dt', text='Voorlopig energielabel').find_next_sibling('dd').text[1]
    except:
        dic['Voorlopig energielabel'] = ""
    try:
        dic['Adress'] = House.find('span', class_ ='object-header-subtitle').text
    except:
        dic['Adress'] = ""   
    try:
        dic['Soort woonhuis'] = House.find('div', class_='object-primary').find('dt', text='Soort woonhuis').find_next_sibling('dd').text[:-12]
    except:
        dic['Soort woonhuis'] = ""     
    if i == 0:
        HouseData.writerow(dic.keys())
        
    HouseData.writerow(dic.values())

In [38]:
#Open CSV File in the form of a list and allow for subsetting, move from 0 to the total lenght of the url list len(linkelist).
#Remove old HouseInformation.csv
#try:
#    not(os.path.exists('HouseInformation.csv'))
#except:  
#    os.remove('HouseInformation.csv')

#Set Starting IP Adress:
#cm.new_identity()
    
#Append new information to csv in order to prevent loss of data during ip ban or filehandling errors
with open('HouseInformation.csv', 'a', newline='') as csvfile:
    HouseData = csv.writer(csvfile)
    averagetime = []
    
    ###IMPORTANT### Set the number to scrape
    numbertoscrape = len(linkelist)
    
    #Scraper loop
    for i in range(48039,numbertoscrape):
        try:
            start = datetime.now()
            response = requests.get('http://www.funda.nl'+linkelist[i], headers=headers).text
            House = BeautifulSoup(response, 'html.parser')
            scrapehouse(i)

            #IP Randomizer
            if i % 100 == 0:
                cm.new_identity()

            #Calculate Time
            end = datetime.now()
            averagetime += [(end - start).total_seconds()]
            meantime = np.mean(averagetime)
            timeleft = str(datetime.timedelta(seconds=(numbertoscrape-i * meantime)))

            #print percentage and time remaining
            print(str(round((i/numbertoscrape)*100,2))+'% Complete' + ', Time remaining: ' + str(timeleft)+ ", Scraping on IP adress "+ str(cm.new_ip)[2:15],  end="\r")
        except:
            cm.new_identity()
    
print("100.0% Complete, Scraping process complete in"+str(time.strftime("%H:%M:%S", time.gmtime(sum(averagetime)))))

NameError: name 'e' is not defined